In [1]:
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

from model import KeywordSpottingModel
from data_loader import load_speech_commands_dataset #TFDatasetAdapter
from utils import set_memory_GB,print_model_size, log_to_file, plot_learning_curves
from augmentations import add_time_shift_noise_and_align, add_noise
from train import trainig_loop






/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:1045: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type=

In [2]:
torch.cuda.is_available()

True

In [3]:
set_memory_GB(1)

Memory fraction set to 0.022458079576498518
Memory fraction in GB: 1.0


In [4]:
train_ds, val_ds, test_ds, info = load_speech_commands_dataset()

2024-08-28 09:52:29.606248: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-28 09:52:29.613559: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-28 09:52:29.631810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 09:52:29.661887: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 09:52:29.670869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attemptin

In [5]:
# maintain seed for repructablity
np.seed = 42
# tf.random.set_seed(42)
torch.manual_seed(0)

In [6]:
label_names = info.features['label'].names[:10]
print(label_names)

['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes']


In [7]:
augmentations = [
    lambda x: add_time_shift_noise_and_align(x),
    lambda x: add_noise(x,noise_level = 0.01)
]

In [8]:
from librosa.feature import mfcc, delta
from torch.utils.data import Dataset
# Define the dataset adapter:
class TFDatasetAdapter(Dataset):
    def __init__(self, tf_dataset, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations=None):
        self.tf_dataset = tf_dataset
        self.data = list(tf_dataset)
        self.fixed_length = fixed_length
        self.n_mfcc = n_mfcc
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.augmentations = augmentations

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio, label = self.data[idx]
        audio = audio.numpy()

        #convert to float
        audio = audio.astype(np.float32)

        # Ensure the audio tensor has the correct shape (1D array)
        if audio.ndim > 1:
            audio = np.squeeze(audio)
            
        # Apply augmentations if any
        if self.augmentations:
            for aug in self.augmentations:
                audio = aug(audio)

        # Pad or trim the audio to the fixed length
        if len(audio) < self.fixed_length:
            audio = np.pad(audio, (0, self.fixed_length - len(audio)), mode='constant')
        else:
            audio = audio[:self.fixed_length]

        # Create MFCCs from an audio tensor using Librosa.
        audio = audio.astype(np.float32)
        MFCC = mfcc(y=audio, sr=16000, n_mfcc=self.n_mfcc, n_fft=self.n_fft, hop_length=self.hop_length, n_mels=self.n_mels)

        # Create MFCC second derivative, first derivative and zeroth order coefficients
        MFCC = mfcc(y=audio, sr=16000, n_mfcc=self.n_mfcc, n_fft=self.n_fft, hop_length=self.hop_length, n_mels=self.n_mels)
        MFCC_delta = delta(MFCC)
        MFCC_delta2 = delta(MFCC, order=2)

        # Stack the three MFCCs together
        MFCC = np.vstack([MFCC, MFCC_delta, MFCC_delta2])

        # Remove extra dimension if it exists
        if MFCC.ndim == 3:
            MFCC = MFCC.squeeze(-1)

        return torch.tensor(MFCC, dtype=torch.float32), torch.tensor(label.numpy(), dtype=torch.long)


In [9]:
# Convert the TFDS dataset to a PyTorch Dataset
fixed_length = 16000
n_mfcc = 13
n_fft = 640
hop_length = 320
n_mels = 40
pytorch_train_dataset = TFDatasetAdapter(train_ds, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations)
pytorch_val_dataset = TFDatasetAdapter(val_ds, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations=None)

2024-08-28 09:52:50.383336: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-08-28 09:52:54.988323: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
# Create a DataLoader to feed the data into the model
batch_size = 16
train_loader = DataLoader(pytorch_train_dataset, batch_size=batch_size, shuffle=True,num_workers=4)
val_loader = DataLoader(pytorch_val_dataset, batch_size=batch_size, shuffle=False,num_workers=4)

In [11]:
for audio, label in train_loader:
    print(audio.shape, label.shape)
    break


torch.Size([16, 39, 51]) torch.Size([16])


# Compute model size

In [20]:
# Initialize model, loss function, and optimizer
input_dim = 39  # Number of MFCC features
d_model = 51  # Number of frames
d_state = 16
d_conv = 4
expand = 2

model = KeywordSpottingModel(input_dim=input_dim, d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand, label_names=label_names, num_mamba_layers=2).to("cuda")


In [21]:

# # Register custom operation
inputs = torch.randn(batch_size, input_dim, d_model).to("cuda")
        
# macs, params, ret_layer_info = thop.profile(model, inputs=(torch.randn(batch_size, 13, 101).to("cuda"),)
# ,custom_ops={Mamba: calculate_MAMBA_flops},report_missing=True, ret_layer_info=True)
# print()
# print(f"MACs: {macs} Which are {macs/1e9} Giga-MACs, Params: {params}")

print_model_size(model,input_size=inputs)

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Customize rule calculate_MAMBA_flops() <class 'mamba_ssm.modules.mamba_simple.Mamba'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.

MACs: 39179832.0 Which are 0.039179832 Giga-MACs, Params: 46624.0



(39179832.0, 46624.0)

# Training loop

# With L2 regulariztion AND Droput layer

In [22]:
criterion = nn.CrossEntropyLoss().to("cuda")
base_optimizer = optim.Adam(model.parameters(), lr=0.0024, weight_decay=2.80475e-05) # weight_decay for L2 regulariztopn

from torch_optimizer import Lookahead

optimizer = Lookahead(base_optimizer, k=5, alpha=0.5)  # Wrap around Adam


In [23]:
import torch.optim as optim
# Adding learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

In [24]:
num_epochs = 100
train_accuracies, val_accuracies, train_losses, val_losses = trainig_loop(model, num_epochs, train_loader, val_loader, criterion, optimizer, scheduler)

100%|██████████████████████████████████████████████████████████████████| 1924/1924 [02:06<00:00, 15.18it/s]

Epoch 1/100, Training Loss: 1.3429627591616276, Training Accuracy: 52.10439078293087%


Validation Loss: 0.8243570596236607, Validation Accuracy: 73.02187415608965%
Learning rate after epoch 1: [0.0024]
Best model saved


100%|██████████████████████████████████████████████████████████████████| 1924/1924 [01:59<00:00, 16.05it/s]

Epoch 2/100, Training Loss: 0.6896386833632996, Training Accuracy: 76.46657349930125%


...

Epoch 27/100, Training Loss: 0.20877488850868353, Training Accuracy: 92.65169488771166%


Validation Loss: 0.25764946759967455, Validation Accuracy: 92.24952741020795%
Learning rate after epoch 27: [0.0024]


100%|██████████████████████████████████████████████████████████████████| 1924/1924 [00:52<00:00, 36.33it/s]

Epoch 28/100, Training Loss: 0.20900268197980426, Training Accuracy: 92.70694530208976%


Validation Loss: 0.2511473190448857, Validation Accuracy: 91.9524709694842%
Learning rate after epoch 28: [0.0024]
Early stopping triggered
Stopping training at epoch 28 due to early stopping


In [ ]:
# Plot the learning curves
plot_learning_curves(train_accuracies, val_accuracies, train_losses, val_losses)

In [ ]:
# calc weights for unbalanced dataset
from utils import compute_label_distribution
label_distribution = compute_label_distribution(train_ds)
# Convert label counts to a list of counts in the order of label indices
counts_list = [label_distribution[i] for i in sorted(label_distribution.keys())]
# Compute total number of samples
total_samples = sum(counts_list)

# Compute the number of classes
num_classes = len(counts_list)

# Calculate class weights
class_weights = [total_samples / (num_classes * count) for count in counts_list]




In [ ]:
class_weights_normalized = class_weights / np.sum(class_weights)  # Normalize class weights
# print(class_weights_normalized)
class_weights_tensor = torch.tensor(class_weights_normalized).to("cuda")
# fine tune the model with balanced dataset
max_samples = int(len(pytorch_train_dataset) * min(class_weights_normalized) *len(class_weights_normalized))
sampler = torch.utils.data.WeightedRandomSampler(
    weights=class_weights_tensor,
    num_samples=len(pytorch_train_dataset),
    replacement=True
)
train_loader = DataLoader(pytorch_train_dataset, batch_size=batch_size,num_workers=4,prefetch_factor=2,sampler=sampler)


In [ ]:
model = KeywordSpottingModel(input_dim=input_dim, d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand, label_names=info.features['label'].names).to("cuda")
model.load_state_dict(torch.load("best_model.pth"),strict=False)
criterion = nn.CrossEntropyLoss()
base_optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=1e-6) # weight_decay for L2 regulariztopn
optimizer = Lookahead(base_optimizer, k=5, alpha=0.5)  # Wrap around Adam
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

In [ ]:
num_epochs = 10
train_accuracies, val_accuracies, train_losses, val_losses = trainig_loop(model, num_epochs, train_loader, val_loader, criterion, optimizer, scheduler, save_best_model=False)

In [25]:
# load test data
# model.load_state_dict(torch.load("best_model.pth"))
pytorch_test_dataset = TFDatasetAdapter(test_ds, fixed_length, n_mfcc, n_fft, hop_length, n_mels, augmentations=None)
test_loader = DataLoader(pytorch_test_dataset, batch_size=batch_size, shuffle=False,num_workers=4,prefetch_factor=2)
# model.load_state_dict(torch.load("best_model.pth"),strict=False)
# Evaluate the model on the test set
accuracy = 0
total = 0
model.eval()

with torch.no_grad():
    for audio, labels in test_loader:
        audio, labels = audio.to("cuda"), labels.to("cuda")
        outputs = model(audio)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        accuracy += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * accuracy / total}%')


Test Accuracy: 91.40893470790378%


In [26]:
def compute_inference_GPU_mem(model, input):
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    m0 = torch.cuda.max_memory_allocated()
    model(input)
    m1 = torch.cuda.max_memory_allocated()
    # Compute total memory used
    total_memory = (m1 - m0) / 1e6  # Convert to MB
    
    return total_memory

In [27]:
import pandas as pd
# from utils import compute_inference_GPU_mem
#save model size(macs, params) and accuracy
macs, params = print_model_size(model,input_size=inputs)
macs = macs/1e9
accuracy = max(val_accuracies)
data = {'Model': ['KeywordSpottingModel_RSM_Norm_0-1-2_order'], 'GMACs': [macs], 'Params': [params], 'Accuracy': [accuracy]}
model_config = {'input_dim': input_dim, 'd_model': d_model, 'd_state': d_state, 'd_conv': d_conv, 'expand': expand}
data.update(model_config)
inf_GPU_mem = compute_inference_GPU_mem(model, input=torch.randn(1, input_dim, d_model).to("cuda"))
#inference macs and params
inf_macs, inf_params = print_model_size(model,input_size=torch.randn(1, input_dim, d_model).to("cuda"))
inference_data = {'Inference CUDA Mem in MB': [inf_GPU_mem], 'Inference GMACs': [inf_macs/1e9], 'Inference Params': [inf_params]}
data.update(inference_data)
df = pd.DataFrame(data, index=[0])
df.to_csv('results.csv', mode='a', header=False)


[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Customize rule calculate_MAMBA_flops() <class 'mamba_ssm.modules.mamba_simple.Mamba'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.

MACs: 39179832.0 Which are 0.039179832 Giga-MACs, Params: 46624.0

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Customize rule calculate_MAMBA_flops() <class 'mamba_ssm.modules.mamba_simple.Mamba'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.

MACs: 3941637.0 Which are 0.003941637 Giga-MACs, Params: 46624.0



In [ ]:
import pandas as pd

results_file = pd.read_csv('results.csv')

memory_usage = []

for index, row in results_file.iterrows():
    
    input_dim = row['input_dim']
    d_model = row['d_model']
    d_state = row['d_state']
    d_conv = row['d_conv']
    expand = row['expand']

    model = KeywordSpottingModel(input_dim=input_dim, d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand, label_names=info.features['label'].names).to("cuda")
    input = torch.randn(1, input_dim, d_model).to("cuda")

    # inf_GPU_mem = compute_inference_GPU_mem(model, input=torch.randn(1, input_dim, d_model).to("cuda"))
    # #inference macs and params
    inf_macs, inf_params = print_model_size(model,input_size=torch.randn(1, input_dim, d_model).to("cuda"))
    inference_data = {'Inference GMACs': [inf_macs/1e9], 'Inference Params': [inf_params]}
    print(inference_data)
# add inference data to the DataFrame
    results_file.at[index, 'Inference GMACs'] = inference_data['Inference GMACs'][0]
    results_file.at[index, 'Inference Params'] = inference_data['Inference Params'][0]

# Save the updated DataFrame back to the CSV file
results_file.to_csv('results.csv', index=False)
    






